In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from keras.models import Sequential
from keras.applications.vgg19 import VGG19
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import layers, models, optimizers
from keras.layers import Flatten, Dense, Input

In [ ]:
(train_images, train_labels),(test_images, test_labels) = cifar10.load_data()

170498071/170498071 [==============================] - 8s 0us/step


In [ ]:
print("train_image",train_images.shape)
print("train_label",train_labels.shape)
print("test_image",test_images.shape)
print("test_label",test_labels.shape)
print(train_images.dtype)

train_image (50000, 32, 32, 3)
train_label (50000, 1)
test_image (10000, 32, 32, 3)
test_label (10000, 1)
uint8


In [ ]:
def resize_image(img):
  num_image = img.shape[0]
  print(num_image)
  new_array = np.zeros((num_image,48,48,3))
  for i in range(num_image):
    new_array[i] = cv2.resize (img[i, :, :, :],(48,48))
  return new_array

In [ ]:
train_images = resize_image(train_images)
test_images = resize_image(test_images)

50000
10000


In [ ]:
print("train_image", train_images.shape)
print("test_image", test_images.shape)

train_image (50000, 48, 48, 3)
test_image (10000, 48, 48, 3)


In [ ]:
print(train_labels)

[[6]
 [9]
 [9]
 ...
 [9]
 [1]
 [1]]


In [ ]:
train_labels = to_categorical(train_labels,num_classes=10)
test_labels = to_categorical(test_labels,num_classes=10)

In [ ]:
print(train_labels)

[[6]
 [9]
 [9]
 ...
 [9]
 [1]
 [1]]


In [ ]:
print("train_label",train_labels.shape)
print("test_label",test_labels.shape)

train_label (50000, 10)
test_label (10000, 10)


In [ ]:
vgg = tf.keras.applications.VGG19(weights = "imagenet" ,include_top=False, input_shape = (48,48,3))

80134624/80134624 [==============================] - 3s 0us/step


In [ ]:
model = models.Sequential()

for layers in vgg.layers:
  model.add(layers)

for  layers in model.layers:
  layers.trainable = False

model.add(Flatten())
model.add(Dense(128,activation = 'relu'))
model.add(Dense(10, activation = 'softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 48, 48, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 48, 48, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 24, 24, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 24, 24, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 24, 24, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 12, 12, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 12, 12, 256)       2

In [ ]:
# tf.keras.optimizers.Adam(learning_rate=0.001, beta_1 = 0.9,beta_2 = 0.99, epsilon=None)

In [ ]:
model.compile(optimizer="Adam",loss="categorical_crossentropy",metrics=["accuracy"])

In [ ]:
hist = model.fit(train_images, train_labels, validation_split = 0.15, epochs = 20, batch_size = 1000)

In [ ]:
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Initialize the base VGG19 model
vgg = tf.keras.applications.VGG19(weights="imagenet", include_top=False, input_shape=(48, 48, 3))
vgg.trainable = False

model = tf.keras.Sequential()

for layer in vgg.layers:
    model.add(layer)

model.add(Flatten())
model.add(Dense(256, activation='relu', kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.15  # Assuming your dataset has a 15% validation split
)

# Fit the model
hist = model.fit(
    datagen.flow(train_images, train_labels, subset='training', batch_size=32),
    validation_data=datagen.flow(train_images, train_labels, subset='validation'),
    epochs=50,
    callbacks=[early_stopping, reduce_lr]
)
